In [1]:
import pandas as pd
import os
import ast
import numpy as np
from Bio.SeqUtils.ProtParam import ProteinAnalysis
pd.options.display.max_colwidth = 100
pd.options.display.float_format = '{:.2f}'.format

In [2]:
all_results = []
for f in os.listdir('.'):
    if '.data' in f:
        with open(f, 'r') as g:
            all_results.append(ast.literal_eval(g.read()))

In [3]:
def standardize_to_uM(concentration, unit, sequence):
    concentration = concentration.replace(' ', '')
    try:
        concentration = float(concentration)
    except:
        return None
    if unit == 'uM' or unit == u'\xb5M' or unit == u'uM)':
        return concentration
    elif unit == 'ug/ml' or unit == u'\xb5g/ml' or unit == u'ug/ml)':
        try:
            molWt = ProteinAnalysis(sequence).molecular_weight()
        except ValueError:
            return None
        return concentration * 1000/molWt
    elif unit == 'nmol/g' or unit == 'pmol/mg':
        #1g, at density of 1g/mL, is 1mL, so nmol/g is nmol/mL = umol/L = uM yay!
        return concentration
    else:
        # print 'Unit not recognized: ' + unit
        return None

In [4]:
def convert_result_to_rows(sequence, result):
    rows = []
    if 'bacteria' not in result:
        return rows
    for bacterium, strain in result['bacteria']:
        
        rows.append({
            'bacterium': bacterium,
            'strain': strain,
            'sequence': sequence.upper(),
            'url_source': result['url_sources'][0],
            'value': standardize_to_uM(
                result['bacteria'][(bacterium, strain)]['value'],
                result['bacteria'][(bacterium, strain)]['unit'],
                sequence
            ),
            'modifications': result['modifications'] if 'modifications' in result else [],
            'unit': 'uM'
        })
        if rows[-1]['value']:
            rows[-1]['value'] = np.log10(rows[-1]['value'])
    return rows

In [5]:
rows = []
for result_set in all_results:
    for sequence in result_set:
        for row in convert_result_to_rows(sequence, result_set[sequence]):
            rows.append(row)

In [6]:
df = pd.DataFrame(rows)

In [7]:
def is_modified(modifications_list):
    return len(modifications_list) > 0

df['is_modified'] = df.modifications.apply(is_modified)

In [8]:
def has_non_cterminal_modification(modifications_list):
    return any(['C-Term' not in modification for modification in modifications_list])

df['has_non_cterminal_modification'] = df.modifications.apply(has_non_cterminal_modification)
#df['has_non_cterminal_modification'] = df.groupby(['sequence'])['has_non_cterminal_modification'].transform(max)

df['has_cterminal_modification'] = df.is_modified & ~df.has_non_cterminal_modification
#df['has_cterminal_modification'] = df.groupby(['sequence'])['has_cterminal_modification'].transform(max)

In [9]:
# Clean sequences
df.sequence = df.sequence.str.strip()
df = df.loc[df.sequence != '/']

In [10]:
# Exclude sequences with modifications

# Exclude rows from YADAMP and CAMP for having no modification data

#     Unless that sequence is in another DB

In [11]:
df = df.loc[df.has_non_cterminal_modification == False]

no_modification_data_sources = ['camp3', 'yadamp']

def datasource_has_modifications(cell):
    # Everything except CAMP and YADAMP has modification data
    return not any([s in cell for s in no_modification_data_sources])

df['_datasource_has_modifications'] = df['url_source'].apply(datasource_has_modifications)

sequences_containing_modifications = set(df.loc[df._datasource_has_modifications == True, 'sequence'])
def sequence_has_modification_data(cell):
    # If the sequence is labeled modifictationless in another database it's OK
    return cell in sequences_containing_modifications

df['_sequence_has_modifications'] = df['sequence'].apply(sequence_has_modification_data)

df['modification_verified'] = df['_sequence_has_modifications'] | df['_datasource_has_modifications']

df = df.loc[df.modification_verified == True]

In [164]:
character_dict = set([character for sequence in df.sequence for character in sequence])
max_sequence_length = int(df.sequence.str.len().describe(percentiles=[0.95])['95%'])
character_to_index = {
    character: i
    for i, character in enumerate(character_dict)
}

def sequence_to_vector(sequence):
    default = np.zeros([max_sequence_length, len(character_to_index)])
    for i, character in enumerate(sequence[:max_sequence_length]):
        default[i][character_to_index[character]] = 1
    return default

In [151]:
ecoli_df = df.loc[df.bacterium.str.contains('E. coli')].groupby(['sequence', 'bacterium'])
ecoli_df = ecoli_df.mean().reset_index().dropna()

In [152]:
cterminal_amidation = np.array(ecoli_df.has_cterminal_modification)

In [251]:
vectors = ecoli_df['sequence'].apply(sequence_to_vector)
vectors = np.array([list(vector) for vector in vectors]).reshape([-1, max_sequence_length, len(character_to_index)])

In [252]:
labels = np.array(ecoli_df.value)

In [253]:
vectors.shape

(4508, 46, 23)

In [211]:
average = np.mean(labels)
squared_errors = sum([(label - average) ** 2 for label in labels])
baseline_error = squared_errors/len(labels)
print(baseline_error)

0.60182344038256


In [292]:
import keras
from keras.layers import Dense, Dropout, LSTM, Conv2D, Conv1D, MaxPooling1D, MaxPooling2D, Flatten
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [269]:
def baseline_model():
    model = keras.models.Sequential()
    model.add(LSTM(
        128,
        input_shape=(max_sequence_length, len(character_to_index)),
    ))
    """model.add(Dense(
        max_sequence_length * len(character_to_index),
        input_dim = max_sequence_length * len(character_to_index),
        kernel_initializer='normal',
        activation='relu'
    ))"""
    #model.add(Dropout(0.5))
    model.add(Dense(50, kernel_initializer='normal'))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [334]:
def conv_model():
    model = keras.models.Sequential()
    model.add(Conv1D(
        64,
        kernel_size = 5,
        strides = 1,
        activation = 'relu',
        input_shape = (max_sequence_length, len(character_to_index))
    ))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    #model.add(Dropout(0.5))
    model.add(Conv1D(64, 5, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(20, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [270]:
model = baseline_model()

In [271]:
cutoff = int(0.9 * len(labels))
train_x = vectors[:cutoff]
train_y = labels[:cutoff]
test_x = vectors[cutoff:]
test_y = labels[cutoff:]

In [272]:
model.fit(train_x, train_y, batch_size=40, epochs=100)

Epoch 1/100
4057/4057 [==============================] - 6s 1ms/step - loss: 0.7355
Epoch 2/100
4057/4057 [==============================] - 4s 976us/step - loss: 0.5843
Epoch 3/100
4057/4057 [==============================] - 4s 998us/step - loss: 0.5820
Epoch 4/100
4057/4057 [==============================] - 4s 1ms/step - loss: 0.5345
Epoch 5/100
4057/4057 [==============================] - 4s 1ms/step - loss: 0.4627
Epoch 6/100
4057/4057 [==============================] - 4s 1ms/step - loss: 0.4550
Epoch 7/100
4057/4057 [==============================] - 4s 1ms/step - loss: 0.4300
Epoch 8/100
4057/4057 [==============================] - 4s 1ms/step - loss: 0.4261
Epoch 9/100
4057/4057 [==============================] - 5s 1ms/step - loss: 0.4205
Epoch 10/100
4057/4057 [==============================] - 4s 1ms/step - loss: 0.4083
Epoch 11/100
4057/4057 [==============================] - 4s 1ms/step - loss: 0.4057
Epoch 12/100
4057/4057 [==============================] - 4s 1ms/step 

4057/4057 [==============================] - 5s 1ms/step - loss: 0.0918
Epoch 97/100
4057/4057 [==============================] - 5s 1ms/step - loss: 0.0860
Epoch 98/100
4057/4057 [==============================] - 4s 1ms/step - loss: 0.0816
Epoch 99/100
4057/4057 [==============================] - 4s 1ms/step - loss: 0.0880
Epoch 100/100
4057/4057 [==============================] - 5s 1ms/step - loss: 0.0862


In [335]:
convmodel = conv_model()
convmodel.fit(train_x, train_y, batch_size=40, epochs=200)

Epoch 1/200
4057/4057 [==============================] - 3s 786us/step - loss: 0.6582
Epoch 2/200
4057/4057 [==============================] - 1s 159us/step - loss: 0.4150
Epoch 3/200
4057/4057 [==============================] - 1s 173us/step - loss: 0.3663
Epoch 4/200
4057/4057 [==============================] - 1s 185us/step - loss: 0.3441
Epoch 5/200
4057/4057 [==============================] - 1s 181us/step - loss: 0.3144
Epoch 6/200
4057/4057 [==============================] - 1s 186us/step - loss: 0.2783
Epoch 7/200
4057/4057 [==============================] - 1s 176us/step - loss: 0.2662
Epoch 8/200
4057/4057 [==============================] - 1s 185us/step - loss: 0.2457
Epoch 9/200
4057/4057 [==============================] - 1s 201us/step - loss: 0.2339
Epoch 10/200
4057/4057 [==============================] - 1s 183us/step - loss: 0.2215
Epoch 11/200
4057/4057 [==============================] - 1s 168us/step - loss: 0.2071
Epoch 12/200
4057/4057 [============================

4057/4057 [==============================] - 1s 149us/step - loss: 0.0599
Epoch 96/200
4057/4057 [==============================] - 1s 154us/step - loss: 0.0600
Epoch 97/200
4057/4057 [==============================] - 1s 153us/step - loss: 0.0603
Epoch 98/200
4057/4057 [==============================] - 1s 154us/step - loss: 0.0586
Epoch 99/200
4057/4057 [==============================] - 1s 149us/step - loss: 0.0552
Epoch 100/200
4057/4057 [==============================] - 1s 147us/step - loss: 0.0558
Epoch 101/200
4057/4057 [==============================] - 1s 151us/step - loss: 0.0560
Epoch 102/200
4057/4057 [==============================] - 1s 155us/step - loss: 0.0581
Epoch 103/200
4057/4057 [==============================] - 1s 148us/step - loss: 0.0557
Epoch 104/200
4057/4057 [==============================] - 1s 152us/step - loss: 0.0587
Epoch 105/200
4057/4057 [==============================] - 1s 155us/step - loss: 0.0536
Epoch 106/200
4057/4057 [=========================

4057/4057 [==============================] - 1s 158us/step - loss: 0.0436
Epoch 189/200
4057/4057 [==============================] - 1s 157us/step - loss: 0.0421
Epoch 190/200
4057/4057 [==============================] - 1s 161us/step - loss: 0.0407
Epoch 191/200
4057/4057 [==============================] - 1s 159us/step - loss: 0.0381
Epoch 192/200
4057/4057 [==============================] - 1s 161us/step - loss: 0.0392
Epoch 193/200
4057/4057 [==============================] - 1s 160us/step - loss: 0.0401
Epoch 194/200
4057/4057 [==============================] - 1s 152us/step - loss: 0.0417
Epoch 195/200
4057/4057 [==============================] - 1s 150us/step - loss: 0.0378
Epoch 196/200
4057/4057 [==============================] - 1s 151us/step - loss: 0.0404
Epoch 197/200
4057/4057 [==============================] - 1s 150us/step - loss: 0.0400
Epoch 198/200
4057/4057 [==============================] - 1s 153us/step - loss: 0.0421
Epoch 199/200
4057/4057 [=====================

In [273]:
model.evaluate(test_x, test_y)

451/451 [==============================] - 1s 2ms/step


0.7278170822861454

In [336]:
convmodel.evaluate(test_x, test_y)

451/451 [==============================] - 1s 2ms/step


0.48967501355909193

In [206]:
model.predict(vectors[3501:3502]), labels[3501]

(array([[0.48488227]], dtype=float32), -0.4082650891994917)

In [210]:
np.sqrt(0.5229434472210003), np.mean(labels), np.

(0.7231482885418455, 1.1360368427613385)

In [142]:
results, np.array([-0.45823179, -0.35380619, -0.47139098, -0.38037349, -0.43953926,
       -0.39756297, -0.47314774, -0.55682546, -0.51937299, -0.55644232]).mean()

(array([-0.46712678, -0.35881006, -0.46925936, -0.35660461, -0.45682324,
        -0.39039968, -0.47390782, -0.55036171, -0.47766049, -0.53645682]),
 -0.4606693190000001)

In [145]:
np.array([-0.46031729, -0.38157843, -0.4546283 , -0.36382549, -0.46000479,
       -0.43178365, -0.46529081, -0.53016265, -0.52939475, -0.54354656]).mean()

-0.462053272

In [ ]:
train_x = 

In [60]:
import tensorflow as tf
with tf.Graph().as_default():
    features = {
        'sequence_vectors': vectors,
        'cterminal_amidation': cterminal_amidation
    }

    feature_columns = [tf.feature_column.numeric_column(key=key) for key in features]

    classifier = tf.estimator.DNNRegressor(
        feature_columns=feature_columns,
        hidden_units=[10, 10],
    )

    def train_input_fn(dataset):
        dataset = tf.data.Dataset.from_tensor_slices((features, labels))
        return dataset.shuffle(5000).repeat().batch(101)

    classifier.train(
        input_fn=lambda:train_input_fn(dataset),
        steps=1
    )

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x10ef69b90>, '_evaluation_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/var/folders/c5/1zp_dmh96q16n8vk97j4jk4c0000gn/T/tmpHO7Ifj', '_save_summary_steps': 100}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


InvalidArgumentError: Input to reshape is a tensor with 106858 values, but the requested shape has 101
	 [[Node: dnn/input_from_feature_columns/input_layer/sequence_vectors/Reshape = Reshape[T=DT_FLOAT, Tshape=DT_INT32, _device="/job:localhost/replica:0/task:0/device:CPU:0"](dnn/input_from_feature_columns/input_layer/sequence_vectors/ToFloat, dnn/input_from_feature_columns/input_layer/sequence_vectors/Reshape/shape)]]

Caused by op u'dnn/input_from_feature_columns/input_layer/sequence_vectors/Reshape', defined at:
  File "/Users/zswitten/.pyenv/versions/2.7.12/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/Users/zswitten/.pyenv/versions/2.7.12/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Users/zswitten/.pyenv/versions/2.7.12/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/zswitten/.pyenv/versions/2.7.12/lib/python2.7/